<font color=black size=5 >**交叉验证Cross validation**</font>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
digits=datasets.load_digits()
X=digits.data
y=digits.target

<font color=black size=5 >**测试train_test_split**</font>

In [3]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4)

In [4]:
from sklearn.neighbors import KNeighborsClassifier

best_score,best_p,best_k=0,0,0
for k in range(2,11):
    for p in range(1,6):
        knn_clf=KNeighborsClassifier(weights='distance',n_neighbors=k,p=p)
        knn_clf.fit(X_train,y_train)
        score=knn_clf.score(X_test,y_test)
        if score>best_score:
            best_score,best_p,best_k=score,p,k
            
print('Best K =',best_k)
print('Best P =',best_p)
print('Best Score =',best_score)

Best K = 3
Best P = 5
Best Score = 0.9860917941585535


<font color=black size=5 >**使用交叉验证**</font>

In [5]:
from sklearn.model_selection import cross_val_score

knn_clf=KNeighborsClassifier()
cross_val_score(knn_clf,X_train,y_train)

array([0.98148148, 0.96759259, 0.97685185, 0.98604651, 0.97674419])

In [6]:
from sklearn.neighbors import KNeighborsClassifier

best_score,best_p,best_k=0,0,0
for k in range(2,11):
    for p in range(1,6):
        knn_clf=KNeighborsClassifier(weights='distance',n_neighbors=k,p=p)
        scores=cross_val_score(knn_clf,X_train,y_train)
        score=np.mean(scores)
        if score>best_score:
            best_score,best_p,best_k=score,p,k
            
print('Best K =',best_k)
print('Best P =',best_p)
print('Best Score =',best_score)

Best K = 8
Best P = 4
Best Score = 0.9814556416881999


In [7]:
#然后使用交叉验证得到的K和P得到模型
best_knn_clf=KNeighborsClassifier(weights='distance',n_neighbors=3,p=4)

In [8]:
best_knn_clf.fit(X_train,y_train)
best_knn_clf.score(X_test,y_test)

0.980528511821975

缺点：每次训练k个模型，相当于整体性能慢了k倍

<font color=black size=5 >**回顾网格搜索**</font>

In [9]:
from sklearn.model_selection import GridSearchCV

param_grid=[
 
    {
        'weights':['distance'],
        'n_neighbors':[i for i in range(1,11)],
        'p':[i for i in range(1,6)]
    }    
]

grid_search=GridSearchCV(knn_clf,param_grid,n_jobs=1,verbose=1)
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:   26.0s finished


GridSearchCV(estimator=KNeighborsClassifier(n_neighbors=10, p=5,
                                            weights='distance'),
             n_jobs=1,
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             verbose=1)

In [10]:
grid_search.best_score_

0.9814556416881999

In [11]:
grid_search.best_params_

{'n_neighbors': 8, 'p': 4, 'weights': 'distance'}

In [12]:
grid_search.best_estimator_

KNeighborsClassifier(n_neighbors=8, p=4, weights='distance')

In [13]:
best_knn_clf=grid_search.best_estimator_
best_knn_clf.score(X_test,y_test)

0.9791376912378303

In [14]:
cross_val_score(knn_clf,X_train,y_train,cv=8)

array([0.98518519, 0.97037037, 0.95555556, 0.97037037, 0.99259259,
       0.98518519, 0.95522388, 0.97761194])

In [15]:
GridSearchCV(knn_clf,param_grid,verbose=1,cv=8)

GridSearchCV(cv=8,
             estimator=KNeighborsClassifier(n_neighbors=10, p=5,
                                            weights='distance'),
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             verbose=1)

<font color=black size=5 >**留一法LOO-CV**</font>

把训练数据集分成m份，将m-1份样本进行训练，用一个样本测试

完全不受随机的影响，最接近模型真正的性能指标

缺点：计算量巨大